## 2. $L^2$ regularization

As we saw previously, overfitting is what happens when variance is too big. Regularization is a way to solve this issue without impeding the network's power too much: the idea is to penalize model complexity by modifying the cost function.


We therefore introduce the following cost:
$$C = C_0 + \frac {\lambda} {2n} \sum_w w^2$$
where $\lambda \in \mathbb{R}+$ is called the **regularization parameter**, and $C_0$ is a non-regularized cost function. For instance if $C_0$ is the cross entropy cost function:
$$C = - \frac {1} {n} \sum_x yln(a) + (1-y)ln(1-a) + \frac {\lambda} {2n} \sum_w w^2$$

And when $C_0$ is the quadratic cost function:
$$C = - \frac {1} {n} \sum_x ||y(x)-a||^2 + \frac {\lambda} {2n} \sum_w w^2$$

$\lambda$ represents whether we are more concerned with bias or variance. Notice that we haven't yet shown how exactly penalizing big weights is tantamount to avoiding excessive model complexity.

Regularization doesn't make gradient descent much more complicated: the rate of change of $C_0$ with respect to the weights and biases can be computed using backpropagation, and the gradient is given by $\frac {\partial C} {\partial w} = \frac {\partial C_0} {\partial w} + \frac {\lambda w} {n}$ and $\frac {\partial C} {\partial b} = \frac {\partial C_0} {\partial b}$. Stochastic gradient descent also happens naturally (for weights, approximate the gradient of $C_0$ and then add the $\frac {\lambda w} {n}$ term, for biases, regularization doen't change anything).

### Testing $L^2$ regularization
Let's test these changes on the neural network that we used in the previous part to showcase the ills of overfitting. As often, this code is Michael Nielsen's.

In [1]:
#### Libraries
# Standard library
import json
import random
import sys

# Third-party libraries
import numpy as np

In [2]:
class QuadraticCost(object):

    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.

        """
        return 0.5*np.linalg.norm(a-y)**2

    @staticmethod
    def delta(z, a, y):
        """Return the error delta from the output layer."""
        return (a-y) * sigmoid_prime(z)

In [3]:
class CrossEntropyCost(object):

    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.  Note that np.nan_to_num is used to ensure numerical
        stability.  In particular, if both ``a`` and ``y`` have a 1.0
        in the same slot, then the expression (1-y)*np.log(1-a)
        returns nan.  The np.nan_to_num ensures that that is converted
        to the correct value (0.0).

        """
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))

    @staticmethod
    def delta(z, a, y):
        """Return the error delta from the output layer.  Note that the
        parameter ``z`` is not used by the method.  It is included in
        the method's parameters in order to make the interface
        consistent with the delta method for other cost classes.

        """
        return (a-y)

In [4]:
#### Main Network class
class Network(object):

    def __init__(self, sizes, cost=CrossEntropyCost):
        """The list ``sizes`` contains the number of neurons in the respective
        layers of the network.  For example, if the list was [2, 3, 1]
        then it would be a three-layer network, with the first layer
        containing 2 neurons, the second layer 3 neurons, and the
        third layer 1 neuron.  The biases and weights for the network
        are initialized randomly, using
        ``self.default_weight_initializer`` (see docstring for that
        method).

        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost=cost

    def default_weight_initializer(self):
        """Initialize each weight using a Gaussian distribution with mean 0
        and standard deviation 1 over the square root of the number of
        weights connecting to the same neuron.  Initialize the biases
        using a Gaussian distribution with mean 0 and standard
        deviation 1.

        Note that the first layer is assumed to be an input layer, and
        by convention we won't set any biases for those neurons, since
        biases are only ever used in computing the outputs from later
        layers.

        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def large_weight_initializer(self):
        """Initialize the weights using a Gaussian distribution with mean 0
        and standard deviation 1.  Initialize the biases using a
        Gaussian distribution with mean 0 and standard deviation 1.

        Note that the first layer is assumed to be an input layer, and
        by convention we won't set any biases for those neurons, since
        biases are only ever used in computing the outputs from later
        layers.

        This weight and bias initializer uses the same approach as in
        Chapter 1, and is included for purposes of comparison.  It
        will usually be better to use the default weight initializer
        instead.

        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda = 0.0,
            evaluation_data=None,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=False,
            monitor_training_cost=False,
            monitor_training_accuracy=False):
        """Train the neural network using mini-batch stochastic gradient
        descent.  The ``training_data`` is a list of tuples ``(x, y)``
        representing the training inputs and the desired outputs.  The
        other non-optional parameters are self-explanatory, as is the
        regularization parameter ``lmbda``.  The method also accepts
        ``evaluation_data``, usually either the validation or test
        data.  We can monitor the cost and accuracy on either the
        evaluation data or the training data, by setting the
        appropriate flags.  The method returns a tuple containing four
        lists: the (per-epoch) costs on the evaluation data, the
        accuracies on the evaluation data, the costs on the training
        data, and the accuracies on the training data.  All values are
        evaluated at the end of each training epoch.  So, for example,
        if we train for 30 epochs, then the first element of the tuple
        will be a 30-element list containing the cost on the
        evaluation data at the end of each epoch. Note that the lists
        are empty if the corresponding flag is not set.

        """
        if evaluation_data: n_data = len(evaluation_data)
        n = len(training_data)
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(
                    mini_batch, eta, lmbda, len(training_data))
            print("Epoch %s training complete" % j)
            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print("Cost on training data: {}".format(cost))
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print("Accuracy on training data: {} / {}".format(
                    accuracy, n))
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print("Cost on evaluation data: {}".format(cost))
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print("Accuracy on evaluation data: {} / {}".format(
                    self.accuracy(evaluation_data), n_data))
            print
        return evaluation_cost, evaluation_accuracy, \
            training_cost, training_accuracy

    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        """Update the network's weights and biases by applying gradient
        descent using backpropagation to a single mini batch.  The
        ``mini_batch`` is a list of tuples ``(x, y)``, ``eta`` is the
        learning rate, ``lmbda`` is the regularization parameter, and
        ``n`` is the total size of the training data set.

        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def accuracy(self, data, convert=False):
        """Return the number of inputs in ``data`` for which the neural
        network outputs the correct result. The neural network's
        output is assumed to be the index of whichever neuron in the
        final layer has the highest activation.

        The flag ``convert`` should be set to False if the data set is
        validation or test data (the usual case), and to True if the
        data set is the training data. The need for this flag arises
        due to differences in the way the results ``y`` are
        represented in the different data sets.  In particular, it
        flags whether we need to convert between the different
        representations.  It may seem strange to use different
        representations for the different data sets.  Why not use the
        same representation for all three data sets?  It's done for
        efficiency reasons -- the program usually evaluates the cost
        on the training data and the accuracy on other data sets.
        These are different types of computations, and using different
        representations speeds things up.  More details on the
        representations can be found in
        mnist_loader.load_data_wrapper.

        """
        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                       for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in data]
        return sum(int(x == y) for (x, y) in results)

    def total_cost(self, data, lmbda, convert=False):
        """Return the total cost for the data set ``data``.  The flag
        ``convert`` should be set to False if the data set is the
        training data (the usual case), and to True if the data set is
        the validation or test data.  See comments on the similar (but
        reversed) convention for the ``accuracy`` method, above.
        """
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
        cost += 0.5*(lmbda/len(data))*sum(
            np.linalg.norm(w)**2 for w in self.weights)
        return cost

    def save(self, filename):
        """Save the neural network to the file ``filename``."""
        data = {"sizes": self.sizes,
                "weights": [w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases],
                "cost": str(self.cost.__name__)}
        f = open(filename, "w")
        json.dump(data, f)
        f.close()


In [5]:
#### Loading a Network
def load(filename):
    """Load a neural network from the file ``filename``.  Returns an
    instance of Network.

    """
    f = open(filename, "r")
    data = json.load(f)
    f.close()
    cost = getattr(sys.modules[__name__], data["cost"])
    net = Network(data["sizes"], cost=cost)
    net.weights = [np.array(w) for w in data["weights"]]
    net.biases = [np.array(b) for b in data["biases"]]
    return net



In [6]:
#### Miscellaneous functions
def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the j'th position
    and zeroes elsewhere.  This is used to convert a digit (0...9)
    into a corresponding desired output from the neural network.

    """
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [7]:
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))


In [8]:
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))


These are what's in ``mnist_loader.py``, *i.e.* the functions that load MNIST the right way:

In [9]:
#### Libraries
# Standard library
import pickle
import gzip

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('NNDL_file/data/mnist.pkl.gz')
    training_data, validation_data, test_data = pickle.load(f,encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

Once again to import the data I had to tinker a little bit with the code so that it would work in Python 3:
* changed ``cPickle`` into ``Pickle``
* converted ``test_data``, ``validation_data``, and ``training_data`` into lists for them to be iterable

In [11]:
training_data, validation_data, test_data = load_data_wrapper()
test_data = list(test_data)
validation_data = list(validation_data)
training_data = list(training_data)

net = Network([784, 30, 10], cost=CrossEntropyCost) 
net.large_weight_initializer()

net.SGD(training_data[:5000], 100, 10, 0.5, evaluation_data=test_data, lmbda = 0.1,monitor_evaluation_cost=True, monitor_evaluation_accuracy=True, monitor_training_cost=True, monitor_training_accuracy=True)


Epoch 0 training complete
Cost on training data: 1.284895665521856
Accuracy on training data: 4197 / 5000
Cost on evaluation data: 1.3106159930648673
Accuracy on evaluation data: 8079 / 10000
Epoch 1 training complete
Cost on training data: 1.0055176989939365
Accuracy on training data: 4419 / 5000
Cost on evaluation data: 1.0855912301437893
Accuracy on evaluation data: 8466 / 10000
Epoch 2 training complete
Cost on training data: 0.8536924837589502
Accuracy on training data: 4548 / 5000
Cost on evaluation data: 0.9843839284571787
Accuracy on evaluation data: 8637 / 10000
Epoch 3 training complete
Cost on training data: 0.7550429847899467
Accuracy on training data: 4621 / 5000
Cost on evaluation data: 0.9094653055005214
Accuracy on evaluation data: 8759 / 10000
Epoch 4 training complete
Cost on training data: 0.6960369940641402
Accuracy on training data: 4665 / 5000
Cost on evaluation data: 0.8945695614612817
Accuracy on evaluation data: 8771 / 10000
Epoch 5 training complete
Cost on tr

Cost on evaluation data: 0.8019259984651419
Accuracy on evaluation data: 9073 / 10000
Epoch 43 training complete
Cost on training data: 0.24750876087574938
Accuracy on training data: 4978 / 5000
Cost on evaluation data: 0.8045629180223859
Accuracy on evaluation data: 9059 / 10000
Epoch 44 training complete
Cost on training data: 0.24400801383099724
Accuracy on training data: 4980 / 5000
Cost on evaluation data: 0.8033741505093722
Accuracy on evaluation data: 9061 / 10000
Epoch 45 training complete
Cost on training data: 0.24063322736834436
Accuracy on training data: 4980 / 5000
Cost on evaluation data: 0.8024106536035334
Accuracy on evaluation data: 9062 / 10000
Epoch 46 training complete
Cost on training data: 0.23730865314686692
Accuracy on training data: 4985 / 5000
Cost on evaluation data: 0.8021904936600364
Accuracy on evaluation data: 9064 / 10000
Epoch 47 training complete
Cost on training data: 0.2351534627384445
Accuracy on training data: 4986 / 5000
Cost on evaluation data: 0

Accuracy on evaluation data: 9112 / 10000
Epoch 85 training complete
Cost on training data: 0.1677323860890745
Accuracy on training data: 4993 / 5000
Cost on evaluation data: 0.7903766079676818
Accuracy on evaluation data: 9116 / 10000
Epoch 86 training complete
Cost on training data: 0.16692858237637298
Accuracy on training data: 4993 / 5000
Cost on evaluation data: 0.7926626533570458
Accuracy on evaluation data: 9116 / 10000
Epoch 87 training complete
Cost on training data: 0.16558755644718656
Accuracy on training data: 4993 / 5000
Cost on evaluation data: 0.7933616229622065
Accuracy on evaluation data: 9105 / 10000
Epoch 88 training complete
Cost on training data: 0.16396295463262814
Accuracy on training data: 4993 / 5000
Cost on evaluation data: 0.7900946853372273
Accuracy on evaluation data: 9113 / 10000
Epoch 89 training complete
Cost on training data: 0.16296866674980262
Accuracy on training data: 4993 / 5000
Cost on evaluation data: 0.7910083092271212
Accuracy on evaluation dat

([1.3106159930648673,
  1.0855912301437893,
  0.9843839284571787,
  0.9094653055005214,
  0.8945695614612817,
  0.8415109596027928,
  0.8204983364947116,
  0.8090360262976025,
  0.8116160868923874,
  0.8034750226751799,
  0.7942653705778933,
  0.8038803916433308,
  0.7770778882762346,
  0.7981207997530511,
  0.7956941230539883,
  0.7940044854286028,
  0.7884098905590371,
  0.7877065958214239,
  0.8017391447527585,
  0.8027758975024939,
  0.7988330503141711,
  0.7863208848437392,
  0.8062766861606575,
  0.8078216521889695,
  0.7928126400822251,
  0.8035341395445394,
  0.8018307906259353,
  0.8031647596743628,
  0.7822509848715086,
  0.7971092685492595,
  0.7907430401115221,
  0.7936968528534871,
  0.8043262167429458,
  0.7945733489275504,
  0.7980267946316202,
  0.794346412040628,
  0.8000039174875897,
  0.8080880198633662,
  0.7984337229858933,
  0.7959351427051463,
  0.8023341130435928,
  0.8010288177221987,
  0.8019259984651419,
  0.8045629180223859,
  0.8033741505093722,
  0.8024106

It works very well, even though it takes a while. But why ? Why are small weights better ? In what sense are they "less complex" ? 

Here's one way of seing this: when weights are small, changing a few inputs by a little bit at random won't have a huge effect on the network's output. This means that the network is robust, that it doen't contort its parameters to fit the data's noise. Note to self: use the word 'procrustean' somewhere here. 

Although Occam's razor is useful in many situations, it isn't a law of nature: sometimes the more complex answer can be correct (*eg* sometimes polynomal interpolation is preferable to linear regression). We apparently don't have a fully fledged theory of why regularization works. 

## 3. Other regularization techniques

* ### $L^1$ regularization

We add a portion of the $L^1$ norm of the weight vector, instead of a portion of the square of the $L^2$ norm. The new cost function is:

$$C = C_0 + \frac \lambda n \sum_w |w|$$

The weights are updated using the derivative of the previous formula (except when it isn't differentiable, in which case the unregularized rule is used):

$$\frac {\partial C} {\partial w} = \frac {\partial C_0} {\partial w} + \frac \lambda n sgn(w)$$

In both $L^1$ and $L^2$ regularization, every iteration reduces the value of the weights, but by a different amount. In $L^2$ regularization, this amount depends on the value of the weight being updated, whereas in $L^1$ regularization, this amount is always the same.

Therefore $L^2$ has a uniformizing effect on the weights: bigger weights get "taxed" more than small weights. On the other hand, $L^1$ changes every weight by the same amount, and leads to a more irregular distribution of weights.

* ### Dropout

Dropout is a completely different technique, that doesn't involve tweaking the cost function. We use stochastic gradient descent, except for every mini-batch only part of the network is trained: some hidden neurons are "ghosted", propagation/backpropagation happens only on the remaining nodes. For every new mini-batch, a new set of hidden nodes is left out.

Intuitively, this is very close to training different networks on the same data and choosing the output as the average of their outputs, or by making them "vote" for the right answer.

Since at some point during the training, neurons will be learning while their neighours are "ghosted", the network is in a way trained to be robust with regards to changes in the output of nearby neurons. This is a good way of avoiding overfitting. Dropout works well.

* ### Artificially expanding the training data

If we add to the original training data pictures obtained by rotating (and/or translating, skewing) training pictures, we get more data (nearly) for free and make our network more robust with respect to these kinds of image transformations. 
Mike says: **"The general principle is to expand the training data by applying operations that reflect real-world variation."**

If we allow too large rotations of MNIST images, the network may end up producing nonsense. For instance, it may start confusing $6$ and $9$.


### An aside on comparing algorithms

An algorithm can be better than another on a certain dataset, and worse on others. So we should be wary of overly spectacular allegations regarding any given technique: for example, some algorithms are extremely efficient only to solve a very restricted set of problems. 

# IV. Initializing the weight distribution 
Up to now, we've initialized or weights and biases at random, using a normal distribution:
$$\forall l, \forall j, \forall k, w^l_{jk} \sim \mathcal{N}(0, 1)$$
$$\forall l, \forall j, b^l_{j} \sim \mathcal{N}(0, 1)$$



### What's the problem with that ? 

If the initial weights all follow this distribution, then the weighted input $z$ of any given neuron in the first layer is quite likely to be very big or very small, $|z| >> 1 $.

For instance, if the input layer contains $1000$ neurons, for an input $x$ such that $x_i = 1$ for $500$ values of $i$ and $x_i = 0$ for the $500$ others, the weighted input $z$ will follow a centered normal distribution with a standard error close to $22.4$:

$$z = \sum^{500}_{i = 1} w_i + b_i \sim \mathcal{N}(0, 501 \simeq 22.4^2)$$

It is therefore quite probable that $|z| >> 1$, meaning that $\sigma'(z) \simeq 0$: as we saw previously in this chapter, this causes a learning slowdown. This kind of argument generalizes to weights in other layers: the idea is the same but the computations get more ugly.

### What's a better choice ?

Let $n_in$ be the number of input weights on a neuron. Then the weights toward that neuron should be initialized following a normal distribution of standard error $\frac {1} {\sqrt n_{in}}$.

This way, with the same input as above, ie $n_{in} = 1000$:
$$z = \sum^{500}_{i = 1} w_i + b_i \sim \mathcal{N}(0, 1 + \sum^{500}_{i = 1} \frac {1}{n_{in}}) = \mathcal{N}(0, 1 + \frac 1 2) = \mathcal{N}(0, \frac 3 2)$$

The standard deviation is $\sqrt{\frac 3 2 }$so that it is far less likely that $|z| >> 1$.

Biases don't matter too much, so we'll keep initializing them according to $\mathcal{N}(0,1)$.

### What does this achieve in practice ?

Our original approach to initialization can be compared to the one given above in `network2.py` file (or the first Jupyter notebook corresponding to this chapter) by using either the `default_weight_initializer` or the `large_weight_initializer` methods.

Though the final accuracy is pretty similar, the network learns much faster when the weights are initialized with this newer method. There are other example of networks where both the speed and the final accuracy are improved.

# V. Choosing hyper parameters

Examples of hyperparameters:
* the learning rate $\eta$
* the regularization parameter $\lambda$
* mini-batch size
* number of neurons
* number of epochs


When a given choice of hyperparameters sucks, it's hard to know which one to change, and what change to make. This section contains a few tips, tricks and heuristics though there isn't a formal consensus on **the** perfect method. In practice hyperparameter choice is often automated, for example by performing grid search in hyperparameter space.

## 1. Simplify things and make sure you're better than chance

Making sure your network is better than chance for a **simpler task** is an important step: testing and tweaking will often be less time consuming as your training set will be smaller, and at least guarantees that you're going in the right direction.

**Simplifying the network** is useful for similar reasons, as is getting **more frequent feedback** on its performance. All these techniques give us leeway to try more things out, some space to experiments. In general, the hyperparameter choice it leads won't do drastically worse with a more complex network tackling a tougher task.

So you'll often have to fumble, changing one parameter after another and progressively improving your results. This is why feedback is so important.

Such is the general idea. Next we'll see some more strategies regarding specific hyperparameters.

## 2. The learning rate $\eta$

If $\eta$ is too:
* big: the cost will oscillate a lot during learning, without really decreasing
* small: the cost will decrease, but slowly

A decent strategy to get to the sweet spot is therefore to increase $\eta$ while the cost is decreasing during the first epochs, until it starts oscillating.


The learning rate is often variable (as it is in other uses of gradient descent). There are many different methods and heuristics (see https://en.wikipedia.org/wiki/Line_search) to choose where along the direction of the gradient to set the netx point. A general idea is to have $\eta$ rather big in the beginning and getting smaller progressively, changing in value when accuracy stops increasing. In this case the learning rate's size can be a condition for stopping: learning ceases whenever it's $x$ times smaller than its initial value.

Using gradient descent to find a good value for $\eta$ at each step is theoretically possible: it involves the minimisation of the function $[h \mapsto C(w - h \frac {\partial C} {\partial w})]$. Assuming this method can be useful for theoretical proofs of convergence, but in practice it's probably too time consuming.

## 3. Number of epochs

The general idea here is to **stop when the network stops improving** (trating from now, we'll assume we're testing the network on the validation data to set hyperparameters). This is also useful to avoid overfitting.

There are subtleties regarding what is meant by "the network stops improving", that should be handled on a case by case basis, for instance according to how much accuracy we are asking relative to speed.

You can use a "no improvement in ten" rule (stop when the accuracy hasn't increased in the ten last epochs) when you're starting to set the hyperparameters, to weed out obviously bad choices. As you narrow down on decent hyperparameter values, it becomes more reasonable to give the network the benefit of the doubt if it doesn't improve much for a few consecutive epochs: maybe the training data was unlucky, maybe the plateau is only temporary...

To implement early stopping in ``network2.py``, we modify the ``SGD`` method in the ``Network`` class: instead of using a ``for`` loop we can use a conditional ``while`` loop, with the condition to check is whether the the current best accuracy is within the $n$ last elements in the  ``evaluation_accuracy`` list. This entails keeping track of the current best accuracy, which can be done through a ``current_best_accuracy`` variable updated if need be at each iteration.

Another plausible rule for early stoppin would be to compare the current accuracy to the average of the $n$ previous accuracies, and to stop if the difference is small enough. In the same spirit, the network could stop when the variance in the distribution of past accuracies is smaller than some small preset parameter $\epsilon > 0$.

## 4. The $L^2$ regularization parameter $\lambda$

First we use the training data to select a good value for $\eta$, without regularization. Then we can use the validation data to find an order of magnitude for $\lambda$, starting at $\lambda = 1.0$ (arbitrary, apparently). Then tweak it a little bit, and then it's back and forth between $\eta$ and $\lambda$.

## 5. The mini-batch size

Small mini-batches (the extreme case being online learning) pros and cons:
* pros: regular updating, the lesser precision isn't a huge issue (Nielsen's inaccurate compass analogy is great)
* cons: doesn't take advantage of the faster, matrix based implementation of SGD.

# VI. Other techniques

## 1. Alternatives to $SGD$

* **Newton's method aka Hessian optimisation** is faster in theory than Gradient descent since it uses second order information on the cost function to determine the next point (it works assuming local convexity). However it involves computing the (huge) Hessian matrix at each step which is expensive. BFGS is a good algorithm that works similarly, but efficiently computes an approximation of the Hessian.

$$w \to w' = w - \eta H^{-1}\nabla C $$

* **Momentum based Gradient Descent** is a modification of Gradient descent with new variables for each weight/bias: velocities. The gradient only has a direct influence on the velocities, and the velocities determine the rates of change of the weights and biases. 

$$v \to v' = \mu v - \eta \nabla C$$
$$w \to w' = w + v' $$

 If $\mu > 1$ velocities and therefore weights will tend to explose.
 If $\mu < 0$ there will be many useless comings and goings.

## 2. Other neuron models
The difference between these is their activation function. Though we can have some domain specific mathematically informed heuristics, the choice of an appropriate neuron model is to a large extent an empirical matter.

* $tanh$ **neuron**

It's more or less the same thing than the sigmoid neuron, since:

$$\forall x \in \mathbb{R}, tanh(x) = \frac {e^{x} - e^{-x}} {e^{-x} + e^{x}} = \frac {1 - e^{-2x}} {1 + e^{-2x}} = \sigma(2x) - [1 - \sigma(2x)] = 2 \times \sigma(2x)-1$$

* **rectified linear neuron**

The activation function here is $[z \mapsto max(0,z)]$. These can be useful for image recognition. It's as universal (in the sense of being able to approximate any function under reasonable conditions) as the sigmoid.